In [1]:
from google.colab import files

# Upload the file
uploaded = files.upload()

Saving cleaned_Ride_Pricing.csv to cleaned_Ride_Pricing.csv


In [2]:
from google.colab import files

# Upload the file
uploaded = files.upload()

Saving cleaned_Delivery_Pricing.csv to cleaned_Delivery_Pricing.csv


In [3]:
import pandas as pd
import numpy as np
import os
import zipfile
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.feature_selection import mutual_info_regression
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
# Create an output directory for saving results
output_dir = "feature_engineering_outputs"
os.makedirs(output_dir, exist_ok=True)

# Load the dataset
data = pd.read_csv('cleaned_Delivery_Pricing.csv')
print(data.columns)


Index(['Delivery_ID', 'Pickup_Latitude', 'Pickup_Longitude',
       'Dropoff_Latitude', 'Dropoff_Longitude', 'Delivery_Distance_km',
       'Package_Weight_kg', 'Delivery_Time_Est_min', 'Order_Value',
       'Platform_Fees', 'Time_of_Day', 'Day_of_Week', 'Weather_Condition',
       'Delivery_Type', 'Traffic_Index', 'Delivery_Location_Type',
       'Discount_Applied', 'Surge_Multiplier', 'Fuel_Cost',
       'Total_Delivery_Fee'],
      dtype='object')


In [9]:
data = pd.read_csv('cleaned_Ride_Pricing.csv')
print(data.columns)

Index(['trip_id', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
       'dropoff_longitude', 'trip_distance_km', 'trip_duration_min',
       'base_fare', 'surge_multiplier', 'time_of_day', 'day_of_week',
       'weather_condition', 'vehicle_type', 'traffic_index',
       'pickup_location_type', 'dropoff_location_type', 'tolls',
       'cancellation_fee', 'passenger_count', 'total_fare'],
      dtype='object')


In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# 1. Data Preparation

# Handle Outliers function
def handle_outliers(df, columns, method='iqr'):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[col] = np.clip(df[col], lower_bound, upper_bound)
    return df

# Numerical columns for both datasets
ride_numerical_columns = ['base_fare', 'trip_distance_km', 'trip_duration_min', 'surge_multiplier',
                          'tolls', 'cancellation_fee', 'passenger_count', 'total_fare']
delhivery_numerical_columns = ['Delivery_Distance_km', 'Package_Weight_kg', 'Delivery_Time_Est_min',
                               'Order_Value', 'Platform_Fees', 'Traffic_Index', 'Discount_Applied',
                               'Surge_Multiplier', 'Fuel_Cost', 'Total_Delivery_Fee']

# Apply outlier handling
data_delhivery = pd.read_csv('cleaned_Delivery_Pricing.csv')

# Load the Ride dataset
data_ride = pd.read_csv('cleaned_Ride_Pricing.csv')
data_ride = handle_outliers(data_ride, ride_numerical_columns)
data_delhivery = handle_outliers(data_delhivery, delhivery_numerical_columns)

# Handle Categorical Variables function
def handle_categorical(df, categorical_columns):
    for col in categorical_columns:
        if col in df.columns:
            # Update the argument to sparse_output=False
            encoder = OneHotEncoder(sparse_output=False, drop='first')
            encoded_data = encoder.fit_transform(df[[col]])
            encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out([col]))
            df = pd.concat([df, encoded_df], axis=1)
            df.drop(columns=[col], inplace=True)
    return df

# Categorical columns for both datasets
ride_categorical_columns = ['time_of_day', 'day_of_week', 'weather_condition', 'vehicle_type',
                            'traffic_index', 'pickup_location_type', 'dropoff_location_type']
delhivery_categorical_columns = ['Time_of_Day', 'Day_of_Week', 'Weather_Condition', 'Delivery_Type',
                                 'Traffic_Index', 'Delivery_Location_Type']

# Apply categorical handling
data_ride = handle_categorical(data_ride, ride_categorical_columns)
data_delhivery = handle_categorical(data_delhivery, delhivery_categorical_columns)

# Handle Missing Values function
def handle_missing_values(df):
    df.fillna(df.median(), inplace=True)
    return df

# Apply missing value handling
data_ride = handle_missing_values(data_ride)
data_delhivery = handle_missing_values(data_delhivery)

# Final datasets ready for use
print("Ride Pricing Data after preparation:")
print(data_ride.head())

print("Delhivery Pricing Data after preparation:")
print(data_delhivery.head())


Ride Pricing Data after preparation:
    trip_id  pickup_latitude  pickup_longitude  dropoff_latitude  \
0 -1.731791         1.127559          1.158664          0.647785   
1 -1.731445         0.767720          1.607694         -1.638451   
2 -1.731098         0.563436          1.711461          0.592824   
3 -1.730752        -0.826775         -1.533989         -1.371910   
4 -1.730405        -0.215387          0.945952          1.312849   

   dropoff_longitude  trip_distance_km  trip_duration_min  base_fare  \
0           0.721469         -0.324701           0.737833  -1.204356   
1          -0.375122          1.150231           1.220548   0.465176   
2          -1.259699         -1.486150          -0.318106   0.409525   
3           1.410801         -0.839966           1.461906  -1.301746   
4           0.189281          0.523265          -0.830991  -0.174811   

   surge_multiplier     tolls  ...  traffic_index_1.6081797162690412  \
0               NaN  1.278903  ...               

In [15]:
# Save the processed data to CSV
data_ride.to_csv('processed_ride_pricing.csv', index=False)
data_delhivery.to_csv('processed_delhivery_pricing.csv', index=False)

# Final datasets ready for use
print("Ride Pricing Data after preparation:")
print(data_ride.head())

print("Delhivery Pricing Data after preparation:")
print(data_delhivery.head())
from google.colab import files

# Download the files
files.download('processed_ride_pricing.csv')
files.download('processed_delhivery_pricing.csv')


Ride Pricing Data after preparation:
    trip_id  pickup_latitude  pickup_longitude  dropoff_latitude  \
0 -1.731791         1.127559          1.158664          0.647785   
1 -1.731445         0.767720          1.607694         -1.638451   
2 -1.731098         0.563436          1.711461          0.592824   
3 -1.730752        -0.826775         -1.533989         -1.371910   
4 -1.730405        -0.215387          0.945952          1.312849   

   dropoff_longitude  trip_distance_km  trip_duration_min  base_fare  \
0           0.721469         -0.324701           0.737833  -1.204356   
1          -0.375122          1.150231           1.220548   0.465176   
2          -1.259699         -1.486150          -0.318106   0.409525   
3           1.410801         -0.839966           1.461906  -1.301746   
4           0.189281          0.523265          -0.830991  -0.174811   

   surge_multiplier     tolls  ...  traffic_index_1.6081797162690412  \
0               NaN  1.278903  ...               

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
# Print column names to verify
print("Ride Pricing Dataset Columns:")
print(data_ride.columns)

print("Delhivery Pricing Dataset Columns:")
print(data_delhivery.columns)


Ride Pricing Dataset Columns:
Index(['trip_id', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
       'dropoff_longitude', 'trip_distance_km', 'trip_duration_min',
       'base_fare', 'surge_multiplier', 'tolls',
       ...
       'traffic_index_1.6313903948207449', 'traffic_index_1.6546010733724483',
       'traffic_index_1.677811751924152', 'traffic_index_1.7010224304758554',
       'traffic_index_1.724233109027559', 'traffic_index_1.7474437875792626',
       'pickup_location_type_Suburban', 'pickup_location_type_Urban',
       'dropoff_location_type_Urban', 'price_per_km'],
      dtype='object', length=179)
Delhivery Pricing Dataset Columns:
Index(['Delivery_ID', 'Pickup_Latitude', 'Pickup_Longitude',
       'Dropoff_Latitude', 'Dropoff_Longitude', 'Delivery_Distance_km',
       'Package_Weight_kg', 'Delivery_Time_Est_min', 'Order_Value',
       'Platform_Fees',
       ...
       'Traffic_Index_1.93', 'Traffic_Index_1.94', 'Traffic_Index_1.95',
       'Traffic_Index_1.96

In [23]:
# 2. Feature Engineering

# Mathematical Transformations
# For Ride Pricing, assuming 'traffic_index' columns are relevant for 'travel_difficulty'
data_ride['travel_difficulty'] = 0.5 * data_ride['surge_multiplier'] + 0.5 * data_ride['tolls']

# For Delhivery Pricing, assuming 'Traffic_Index' columns are relevant for 'travel_difficulty'
data_delhivery['travel_difficulty'] = 0.5 * data_delhivery['Surge_Multiplier'] + 0.5 * data_delhivery['Platform_Fees']



# For Delhivery Pricing, using available columns
data_delhivery['price_per_km'] = data_delhivery['Order_Value'] / data_delhivery['Delivery_Distance_km']



# Time-Based Features (if 'timestamp' column exists)
if 'timestamp' in data_ride.columns:
    data_ride['timestamp'] = pd.to_datetime(data_ride['timestamp'])
    data_ride['hour'] = data_ride['timestamp'].dt.hour
    data_ride['day_of_week'] = data_ride['timestamp'].dt.dayofweek
    data_ride['is_weekend'] = (data_ride['day_of_week'] >= 5).astype(int)

if 'timestamp' in data_delhivery.columns:
    data_delhivery['timestamp'] = pd.to_datetime(data_delhivery['timestamp'])
    data_delhivery['hour'] = data_delhivery['timestamp'].dt.hour
    data_delhivery['day_of_week'] = data_delhivery['timestamp'].dt.dayofweek
    data_delhivery['is_weekend'] = (data_delhivery['day_of_week'] >= 5).astype(int)

# Interaction Features
data_ride['distance_demand'] = data_ride['trip_distance_km'] * data_ride['total_fare']
data_ride['duration_per_km'] = data_ride['trip_duration_min'] / data_ride['trip_distance_km']

data_delhivery['distance_demand'] = data_delhivery['Delivery_Distance_km'] * data_delhivery['Total_Delivery_Fee']
data_delhivery['duration_per_km'] = data_delhivery['Delivery_Time_Est_min'] / data_delhivery['Delivery_Distance_km']

# View the new features added in both datasets
print("Updated Ride Pricing Data with new features:")
print(data_ride.head())

print("Updated Delhivery Pricing Data with new features:")
print(data_delhivery.head())


Updated Ride Pricing Data with new features:
    trip_id  pickup_latitude  pickup_longitude  dropoff_latitude  \
0 -1.731791         1.127559          1.158664          0.647785   
1 -1.731445         0.767720          1.607694         -1.638451   
2 -1.731098         0.563436          1.711461          0.592824   
3 -1.730752        -0.826775         -1.533989         -1.371910   
4 -1.730405        -0.215387          0.945952          1.312849   

   dropoff_longitude  trip_distance_km  trip_duration_min  base_fare  \
0           0.721469         -0.324701           0.737833  -1.204356   
1          -0.375122          1.150231           1.220548   0.465176   
2          -1.259699         -1.486150          -0.318106   0.409525   
3           1.410801         -0.839966           1.461906  -1.301746   
4           0.189281          0.523265          -0.830991  -0.174811   

   surge_multiplier     tolls  ...  traffic_index_1.7010224304758554  \
0               NaN  1.278903  ...       

In [24]:
from google.colab import files

# Save the datasets as CSV files
data_ride.to_csv('processed_ride_pricing.csv', index=False)
data_delhivery.to_csv('processed_delhivery_pricing.csv', index=False)

# Download the files
files.download('processed_ride_pricing.csv')
files.download('processed_delhivery_pricing.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import pandas as pd
from google.colab import files

# 3. Feature Transformation

# Define the actual numerical columns for both datasets
ride_numerical_columns = ['trip_distance_km', 'trip_duration_min', 'base_fare', 'surge_multiplier',
                          'tolls', 'cancellation_fee', 'passenger_count', 'total_fare']
delhivery_numerical_columns = ['Delivery_Distance_km', 'Package_Weight_kg', 'Delivery_Time_Est_min',
                               'Order_Value', 'Platform_Fees', 'Discount_Applied', 'Surge_Multiplier',
                               'Fuel_Cost', 'Total_Delivery_Fee']  # excluding 'Traffic_Index' for now

# Ensure the columns in the datasets match the ones defined
ride_numerical_columns = [col for col in ride_numerical_columns if col in data_ride.columns]
delhivery_numerical_columns = [col for col in delhivery_numerical_columns if col in data_delhivery.columns]

# Creating pipeline for imputation, scaling, and polynomial feature creation
ride_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # Imputation step
    ('scaler', MinMaxScaler()),  # Scaling step
    ('poly', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False))  # Polynomial features
])

delhivery_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler()),
    ('poly', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False))
])

# Apply pipeline to the datasets
ride_poly_features_transformed = ride_pipeline.fit_transform(data_ride[ride_numerical_columns])
delhivery_poly_features_transformed = delhivery_pipeline.fit_transform(data_delhivery[delhivery_numerical_columns])

# Get the feature names after transformation
# The issue was here: get_feature_names_out expects the original columns before the pipeline transformations
# We now get the column names from the imputer's output
ride_poly_feature_names = ride_pipeline.named_steps['poly'].get_feature_names_out(ride_pipeline.named_steps['imputer'].get_feature_names_out(ride_numerical_columns))
delhivery_poly_feature_names = delhivery_pipeline.named_steps['poly'].get_feature_names_out(delhivery_pipeline.named_steps['imputer'].get_feature_names_out(delhivery_numerical_columns))

# Create DataFrame for the transformed features
ride_poly_features = pd.DataFrame(ride_poly_features_transformed, columns=ride_poly_feature_names)
delhivery_poly_features = pd.DataFrame(delhivery_poly_features_transformed, columns=delhivery_poly_feature_names)

# Save polynomial features to CSV
ride_poly_features.to_csv('ride_poly_features.csv', index=False)
delhivery_poly_features.to_csv('delhivery_poly_features.csv', index=False)

# Download the files to local machine
files.download('ride_poly_features.csv')
files.download('delhivery_poly_features.csv')

/usr/local/lib/python3.11/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['surge_multiplier' 'cancellation_fee']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
# 4. Feature Reduction
from sklearn.decomposition import PCA
import pandas as pd
from google.colab import files

# Assuming scaled_data is already prepared from the previous transformations
# Instead of assuming, define and calculate scaled_data

# Choose the appropriate data for scaling (either ride_poly_features or delhivery_poly_features)
# This example uses ride_poly_features, but you might need to adjust based on your data
scaled_data = ride_pipeline.named_steps['scaler'].fit_transform(ride_poly_features) # Using the scaler from the pipeline

# Apply Principal Component Analysis (PCA) for feature reduction
pca = PCA(n_components=0.95)  # Keep 95% of the variance
pca_data = pca.fit_transform(scaled_data)

# Create DataFrame from PCA data
pca_df = pd.DataFrame(pca_data)

# Save PCA features to CSV
pca_file_path = '/content/pca_features.csv'
pca_df.to_csv(pca_file_path, index=False)

# Save the Explained Variance Ratio to CSV
explained_variance = pd.DataFrame({'Explained Variance': pca.explained_variance_ratio_})
explained_variance_file_path = '/content/explained_variance.csv'
explained_variance.to_csv(explained_variance_file_path, index=False)

# Download the files to your local machine
files.download(pca_file_path)
files.download(explained_variance_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Target Column for Predicting Ride Duration: trip_duration_min
Target Column for Fare Prediction: total_fare
Target Column for Surge Prediction: surge_multiplier
Target Column for Traffic Impact Prediction: traffic_index

In [69]:
from google.colab import files
import shutil

# Step 6: Save Mutual Information results to CSV
output_dir = '/content/output'  # Define your output directory

# Ensure the output directory exists
shutil.os.makedirs(output_dir, exist_ok=True)

for target, mutual_info in mutual_info_dict.items():
    mutual_info_series = pd.Series(mutual_info, index=X.columns).sort_values(ascending=False)
    mutual_info_series.to_csv(f"{output_dir}/mutual_info_{target}.csv")

# Step 7: Save the final processed dataset (PCA features + original features)
processed_data = pd.DataFrame(pca_data, columns=[f"pca_{i+1}" for i in range(pca_data.shape[1])])
final_processed_data = pd.concat([data, processed_data], axis=1)
final_processed_data.to_csv(f"{output_dir}/final_processed_data.csv", index=False)

# Download the saved files to local machine
shutil.make_archive(output_dir, 'zip', output_dir)  # Create a zip file of the output directory
files.download(f"{output_dir}.zip")  # Trigger the download


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [70]:
from google.colab import files
import os

# Step 6: Save Mutual Information results to CSV
output_dir = '/content/output'  # Define your output directory

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Save mutual information CSVs
for target, mutual_info in mutual_info_dict.items():
    mutual_info_series = pd.Series(mutual_info, index=X.columns).sort_values(ascending=False)
    file_path = f"{output_dir}/mutual_info_{target}.csv"
    mutual_info_series.to_csv(file_path)
    # Download each CSV file individually
    files.download(file_path)

# Step 7: Save the final processed dataset (PCA features + original features)
processed_data = pd.DataFrame(pca_data, columns=[f"pca_{i+1}" for i in range(pca_data.shape[1])])
final_processed_data = pd.concat([data, processed_data], axis=1)
final_file_path = f"{output_dir}/final_processed_data.csv"
final_processed_data.to_csv(final_file_path)

# Download the final processed dataset CSV
files.download(final_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>